Note, still in the process of experimenting with parameters

Import packages

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pds
from tensorflow import keras
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.keras.optimizers import Adadelta, Nadam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


The model is added with wanted combinations to be tested

In [2]:
def create_model(x_train, y_train, x_test, y_test):

    model = Sequential()
    
    model.add(Dense({{choice([3,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}, input_shape=(19,)))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}))
    model.add(Dense(1, activation={{choice([None, 'relu'])}}))

    model.compile(loss='mae', 
                  optimizer='Adam',
                  metrics=['mae']
                 )
    
    #result = model.fit(x_train,
    #                  y_train,
    ##                  epochs = 5,
    #                  batch_size=10,
    #                  validation_data=[x_test, y_test])
    result = model.fit(x_train, y_train,
                   batch_size={{choice([64, 128])}},
                   epochs=3,
                   verbose=1,
                   validation_split=0.1)
    
    loss_metric = np.amax(result.history['mean_absolute_error']) 
    print('Current epoch:', result.history['mean_absolute_error'])
    return {'loss': loss_metric, 'status': STATUS_OK, 'model': model}

The model needs a function that deliver the data once in its requested format

In [3]:
def data():
    df = pds.read_pickle('ANN_dataset.pkl')
    
    labels = df['time_left']
    #dataset = dataset.drop(columns=['time_left', 'segment_time']) 
    
    # Make segments categorical
    dataset = pds.get_dummies(df['seg'])

    # Combine segments, timestamps and time from journey start

    # Cyclical timestamps 
    dataset['hr_sin'] = np.sin(df.timestamp.dt.hour*(2.*np.pi/24))
    dataset['hr_cos'] = np.cos(df.timestamp.dt.hour*(2.*np.pi/24))

    # Convert to radians befor trigonometric functions
    dataset['dir_sin'] = np.sin(df.direction*(np.pi/180))
    dataset['dir_cos'] = np.cos(df.direction*(np.pi/180))

    dataset['speed'] = (df['speed']-df['speed'].min())/(df['speed'].max()-df['speed'].min())
    dataset['tsjs'] = (df['tsjs']-df['tsjs'].min())/(df['tsjs'].max()-df['tsjs'].min())

    dataset['lat'] = (df['lat']-df['lat'].min())/(df['lat'].max()-df['lat'].min())
    dataset['lon'] = (df['lon']-df['lon'].min())/(df['lon'].max()-df['lon'].min())
    
    # Include journey number to select entire journeys
    dataset['journey'] = df['journey']
    
    
    divider = 0.2
    num_journeys = dataset.journey.unique()[-1]
    num_test_segments = np.int_(np.round(num_journeys*divider))
    msk = np.random.randint(1, num_journeys, num_test_segments)
    mask = dataset['journey'].isin(msk)

    train_input = dataset[~mask]
    test_input = dataset[mask]

    train_labels = labels[~mask]
    test_labels = labels[mask]

    train_data = train_input.drop(columns=['journey'])
    test_data = test_input.drop(columns=['journey'])

    x_train = train_input.drop(columns=['journey'])
    x_test = test_data
    y_train = train_labels
    y_test = test_labels
    
    return x_train, y_train, x_test, y_test

This calls test, by random choice, 35 different combinations of parameters to the model. The number of evaluations could be increased with the cost of it taking a lot more time.

In [4]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=35,
                                      trials=Trials(),
                                      notebook_name="2.\ Extra\ -\ parameter\ optimization"
                                     )

>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pds
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.keras.optimizers import Adadelta, Nadam
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [3,19,38]),
        'activation': hp.choice('activation', [None, 'relu', 'tanh']),
        'Dense_1': hp.choice('Dense_1', [1,19,38]),
        'activation_1': hp.choice('activation_1', [None, 'relu', 'tanh']),
        'Dense_2': hp.choice('Dense_2', [1,19,38

Epoch 3/3
2444208/2444208 [==============================] - 23s 9us/step - loss: 11.8679 - mean_absolute_error: 11.8679 - val_loss: 12.3197 - val_mean_absolute_error: 12.3197
Current epoch: [14.064938982000811, 12.095236708270846, 11.86793007407437]
Train on 2444208 samples, validate on 271579 samples
Epoch 1/3
2444208/2444208 [==============================] - 44s 18us/step - loss: 12.6287 - mean_absolute_error: 12.6287 - val_loss: 12.1053 - val_mean_absolute_error: 12.1053
Epoch 2/3
2444208/2444208 [==============================] - 44s 18us/step - loss: 11.6003 - mean_absolute_error: 11.6003 - val_loss: 11.9101 - val_mean_absolute_error: 11.9101
Epoch 3/3
2444208/2444208 [==============================] - 42s 17us/step - loss: 11.4946 - mean_absolute_error: 11.4946 - val_loss: 11.8452 - val_mean_absolute_error: 11.8452
Current epoch: [12.628712596885695, 11.60030536630136, 11.494641440150511]
Train on 2444208 samples, validate on 271579 samples
Epoch 1/3
2444208/2444208 [==========

2444208/2444208 [==============================] - 65s 27us/step - loss: 44.9257 - mean_absolute_error: 44.9257 - val_loss: 37.8188 - val_mean_absolute_error: 37.8188
Epoch 2/3
2444208/2444208 [==============================] - 62s 25us/step - loss: 37.0512 - mean_absolute_error: 37.0512 - val_loss: 37.1792 - val_mean_absolute_error: 37.1792
Epoch 3/3
2444208/2444208 [==============================] - 61s 25us/step - loss: 36.9789 - mean_absolute_error: 36.9789 - val_loss: 37.1793 - val_mean_absolute_error: 37.1793
Current epoch: [44.92574022346942, 37.05122048552222, 36.97887174137199]
Train on 2444208 samples, validate on 271579 samples
Epoch 1/3
2444208/2444208 [==============================] - 60s 25us/step - loss: 12.5892 - mean_absolute_error: 12.5892 - val_loss: 12.0994 - val_mean_absolute_error: 12.0994
Epoch 2/3
2444208/2444208 [==============================] - 62s 25us/step - loss: 11.6068 - mean_absolute_error: 11.6068 - val_loss: 11.9545 - val_mean_absolute_error: 11.9545

In [5]:
best_run

{'Dense': 2,
 'Dense_1': 1,
 'Dense_2': 1,
 'activation': 1,
 'activation_1': 2,
 'activation_2': 0,
 'activation_3': 1,
 'batch_size': 0}